In [15]:
import math
import numpy as np
import sys
from typing import List, Tuple
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
from lists import Node

print()

In [31]:
def longest_palindrome(s: str) -> str:
    "Return longest palindrome substring from `s`."
    # Uses Manacher's algorithm.
    # Runs in O(N)
    n = len(s) * 2 + 1
    if n == 1:
        return ""
    lps = [0] * n
    lps[1] = 1  # size of the first palindrome
    c = 1  # center of the outer palindrome
    r = 2  # right border of the outer palindrome
    mxs, mxe = 0, 1  # borders of the largest palindrome
    for i in range(2, n):
        # use the mirror length if `i` is between `c` and `r`.
        l = int(i < r and min(lps[2 * c - i], r - i))
        # Try to expand the length (skip filler chars).
        l += (i > l) & (i + l + 2 < n) & (i + l)
        while i > l and i + l + 2 < n and s[(i - l - 1) // 2] == s[(i + l + 1) // 2]:
            l += 2
        # Store the value.
        lps[i] = l
        # Update the max interval.
        if l > mxe - mxs:
            mxs, mxe = (i - l) // 2, (i + l) // 2
        # If the current palindrome expanded beyond the
        # previous one, replace the old by this one.
        if i + l > r:
            c = i
            r = i + l

    print(mxe)
    return s[mxs:mxe]

In [33]:
longest_palindrome("aba")

3


'aba'

In [244]:
int(math.log(5, 10))

0